Project: Echoes of Power
=====

In this problem set, you will use iPython notebook and various python tools to read data and analyze it.

We will use data from the paper 

In [1]:
%pylab inline



Populating the interactive namespace from numpy and matplotlib


In [2]:
utterances = {}
def load_data():
    with open('wikipedia.talkpages.conversations.txt', "r") as f:
        for line in f:
            split = line.split("+++$+++")
#             print len(split)
            try:
                utterances[split[0].strip()] = {"user_id": split[1].strip(),
                                        "talkpage_user": split[2].strip(), 
                                        "conversation_root": split[3].strip(),
                                        "reply_to": split[4].strip(),
                                        "timestamp": split[5].strip(),
                                        "timestamp_unix": split[6].strip(),
#                                       looks like readme has a small mistake                                     
                                        "raw_text": split[7].strip(),
                                        "text": split[8].strip(),        
                                        }
            except IndexError:
                pass
load_data()


In [3]:
# utterances.keys()

In [4]:
conversations = {}
for u_id, utter in utterances.iteritems():
    user_id = utter["user_id"]
    reply_id = utter.get("reply_to")
#     print utterances.get(reply_id)
#     print reply_id
    root_conversation = utter["conversation_root"]
   
#   doubt
    if root_conversation == u_id:
        continue
    if conversations.get(root_conversation) == None:
        conversations[root_conversation] = [u_id]
    else:
        conversations[root_conversation].append(u_id)
        
        
#     if reply_id and utterances.get(reply_id):
#         reply_user_id = utterances[reply_id]["user_id"]
#         if(conversations.get((user_id, reply_user_id)) == None):
#             conversations[utter["conversation_root"]] = [reply_id, u_id]
#         else:
#             conversations[(user_id, reply_user_id)].append(reply_id)
#             conversations[(user_id, reply_user_id)].append(u_id)
conversations
print len(conversations.keys())

125292


In [5]:
count = 0.0
for i in conversations.keys():
    if utterances[i]["user_id"] == utterances[i]["talkpage_user"]:
        count += 1
print count/125292

0.0426763081442


In [6]:
cnt = 0
for i in utterances.keys():
    if(utterances[i]["reply_to"] == i):
        cnt += 1
#     print utterances[i]["reply_to"]
#     pass
cnt

0

In [7]:

#'577840': ['577843', '577842', '577841', '577844']]
print utterances['577843']
print (utterances['577842']['reply_to'])
print utterances['577841']
# print utterances['577844']['reply_to']
print utterances['577840']

{'conversation_root': '577840', 'user_id': 'Zanimum', 'talkpage_user': 'Zanimum', 'timestamp_unix': '1.179627540E09', 'reply_to': '577842', 'timestamp': '2007-05-19 19:19:00', 'text': "::: No, I'll keep it. :) Just wanted to make sure you knew I didn't click the camera myself. Anyway, thanks for noticing my efforts.  --  [[User:Zanimum|Zanimum]] 19:19, 19 May 2007 (UTC)", 'raw_text': "No, I'll keep it. :) Just wanted to make sure you knew I didn't click the camera myself. Anyway, thanks for noticing my efforts.  --"}
577841
{'conversation_root': '577840', 'user_id': 'Zanimum', 'talkpage_user': 'Zanimum', 'timestamp_unix': '1.179626880E09', 'reply_to': '577840', 'timestamp': '2007-05-19 19:08:00', 'text': ': [Blush] I don\'t know if I do... they were taken by other volunteer photographers, that I "commissioned". I\'ve been aranging as many photo shoots as possible, to get this type of image.  --  [[User:Zanimum|Zanimum]] 19:08, 19 May 2007 (UTC)', 'raw_text': '[Blush] I don\'t know if I

In [8]:
def get_other_user(conversation_list, root_user):
    for i in conversation_list:
        other_user = utterances[i]["user_id"]
        if utterances[i]["user_id"] != root_user and other_user != "":
            return other_user

In [9]:
# conversations

In [10]:
conv_list = ['445339','445338','445337','445336','445335','445344','445345','445342','445343','445340','445341']
# '228054': ['228055', '228056']
conv_list = ['228054','228055', '228056']
conv_list = ['320591', '320597', '320595', '320594', '320593', '320592', '320596']

class Node:
    def __init__(self, u_id):
        self.u_id = u_id
        self.child = None
        self.parent = None
#     def add_child(self, child_node=None, parent_node=None):
#         self.child = child_node
#         self.parent = parent_node
#         return True


def get_root(node_list, start_node):
    if start_node.parent == None:
        return start_node
    else:
        parent = start_node.parent
        return get_root(node_list, node_list[parent])


def compute_conversation_hierarchy(conv_list):
    node_list = {}
    
    for i in conv_list:
        node_list[i] = Node(i)
        
    for i in conv_list:
        reply_to = utterances[i]["reply_to"]
        if reply_to != '-1' and reply_to != "initial_post":
            node_list[i].parent = reply_to
            node_list[reply_to].child = i
#           doubt : if a message has more than 1 replies then we are ignosing that here
#                 : 
        
        
    root = get_root(node_list, node_list.values()[0])
    new_list = [root.u_id]
    cur = root
    while(len(new_list) < len(node_list) and cur.child != None):
        cur = node_list[cur.child]
        new_list.append(cur.u_id)
        
    return new_list

print compute_conversation_hierarchy(conv_list)

#     for i in node_list.values():
#         print i.u_id
#         print "its child is %s" % i.child

#         print "its parent is %s" % i.parent

['320591', '320592', '320593', '320594', '320595', '320596', '320597']


In [11]:
import copy

final_conversations = {}

for i, j in conversations.iteritems():
    conv_list = copy.deepcopy(j)
    conv_list.insert(0, i)
    conv_hierarchy = compute_conversation_hierarchy(conv_list)
    B_user = utterances[i]["talkpage_user"]
    A_user = utterances[i]["user_id"]
    if final_conversations.get((A_user, B_user)):
        final_conversations[(A_user, B_user)].append(conv_hierarchy)
    else:
        final_conversations[(A_user, B_user)] = [conv_hierarchy]
#     break
    
# print final_conversations
    

In [12]:
def append_to_left_of(item, li, item_to_insert, conv_list):
    if(item in li):
        index= li.index(item)
        li.insert(index, item_to_insert)
        conv_list.remove

def make_conv_list(conv_list):
    cur_conv = [conv_list[0]]
    for c in range(1,len(conv_list)):
        reply_to = utterances[c]["reply_to"]
        

conv_list = ['445339','445338','445337','445336','445335','445344','445345','445342','445343','445340','445341']

def get_ordered_conversation(conv_list):
    temp = []
    while(len(conv_list) > 0):
        conv_list = []
        make_conv_list(conv_list)
    
    for c in conv_list:
        reply_to = utterances[c]["reply_to"]
        append_to_left_of
        print reply_to
        
        
get_ordered_conversation(conv_list)

IndexError: list index out of range

In [13]:
conv_left = []


In [14]:
conversation_threads = {}
for i, v in conversations.iteritems():
    root_user = utterances[i]["user_id"]
    other_user = get_other_user(v, root_user)
    conversation_threads[(root_user, other_user)] = get_ordered_conversation(v)
conversation_threads

IndexError: list index out of range

In [15]:
import nltk

from nltk.stem.porter import *
from nltk import sent_tokenize,word_tokenize,porter

import csv
from collections import Counter, defaultdict

In [16]:
def word_proc(x, preProcessing_type):
    
    if (preProcessing_type == 0): return x
    if (preProcessing_type == 1): return x.lower()
    if (preProcessing_type == 2): 
        stemmer = PorterStemmer()
        x = x.lower()
        return stemmer.stem(x)

    return x # default 

def create_markers(category):

    file = open(category+'.txt', "r")
    markers = file.read()
    
    tokens = (re.split('[\t\n]', markers.strip()))
            
    return tokens


conjunction_list = create_markers('conjunctions')  

print conjunction_list

['after', 'although', 'and', 'as', 'as far as', 'as how', 'as if', 'as long as', 'as soon as', 'as though', 'as well as', 'because', 'before', 'both', 'but', 'either', 'even if', 'even though', 'for', 'how', 'however', 'if', 'if only', 'in case', 'in order that', 'neither', 'nor', 'now', 'once', 'only', 'or', 'provided', 'rather than', 'since', 'so', 'so that', 'than', 'that', 'though', 'till', 'unless', 'until', 'when', 'whenever', 'where', 'whereas', 'wherever', 'whether', 'while', 'yet']


In [32]:
# code to compute coordination between user a and user b

"""('Tpbradbury', 'Karanacs'): [['357026', '357027', '357028', '357029']], 
    ('Skope', 'Jedi6'):         [['500004', '500005'], ['375918', '375919'], ['500181', '500183'], 
                                 ['500029', '500030', '500031', '500032']]
"""
"""

utterances = {}
                utterances[split[0].strip()] = {"user_id": split[1].strip(),
                                        "talkpage_user": split[2].strip(), 
                                        "conversation_root": split[3].strip(),
                                        "reply_to": split[4].strip(),
                                        "timestamp": split[5].strip(),
                                        "timestamp_unix": split[6].strip(),
#                                       looks like readme has a small mistake                                     
                                        "raw_text": split[7].strip(),
                                        "text": split[8].strip(),        
                                        
"""

marker_list = {}
utterances_tokenized = {}

def tokenize(utterance):
    preProcessing_type = 1
    
    text = utterances[utterance]["text"]
    
    word_list = (text.strip()).split(' ')
    
    tokens = Counter()
    
    
    # print 'text',text
    
    # for word in word_tokenize(text):
    for word in word_list:
        tokens[word_proc(word, preProcessing_type)] += 1

    return tokens


def contains(utterance, marker):
    
    preProcessing_type = 1
    
    if (utterance not in utterances_tokenized):
        utterances_tokenized[utterance] = tokenize(utterance) # returns dictionary with keys as tokens
        
    # print utterances_tokenized

    # print "--------------------------------------"
    
    word_list = marker_list[marker]
    
    for i in range(len(word_list)):
        word = word_proc(word_list[i], preProcessing_type)
        if (word in utterances_tokenized[utterance]):
            return True
    
    return False


""" 
for consistency with prior work, we employed eight of the nine
LIWC-derived categories [41] deemed to be processed by humans
in a generally non-conscious fashion [28]. Our eight markers are
thus: articles, auxiliary verbs, conjunctions, high-frequency ad-
verbs, impersonal pronouns, personal pronouns, prepositions, and
quantifiers (451 lexemes total)
"""    

# input: list of utterance list. Each utterance list is a conversation   
def compute_coordination(conv_list):
    
    marker_list['articles'] =  ['a', 'an', 'the']
    marker_list['personal_pronouns'] = ['i','them','her','you','he','she','it','they','we']
    marker_list['conjunctions'] = create_markers('conjunctions')
    
    
    coordination_list = {}
    
    for marker in marker_list.keys():
        
        a_usage_count = 0
        b_usage_count = 0
        
        b_follows_a_count = 0
        b_utterance_count = 0
        
        for conversation in conv_list:
            for i in range(len(conversation)): #['500029', '500030', '500031', '500032']
                if (i%2 != 0):continue # for skipping B's utterance
                    
                 
                a_utterance = conversation[i]
                b_utterance = None
                if (i+1 < len(conversation)):
                        b_utterance = conversation[i+1]
                        b_utterance_count +=1
            
                a_utterance_has_m = contains(a_utterance, marker)
                
                b_utterance_has_m = False
                if (b_utterance != None):
                    b_utterance_has_m = contains(b_utterance, marker)
                
                if (a_utterance_has_m):
                    a_usage_count +=1
                    
                if (b_utterance_has_m):    
                    b_usage_count +=1
                    
                if (a_utterance_has_m and b_utterance_has_m):
                    b_follows_a_count +=1
       
    
#        print 'marker',marker
#        print 'b_follows_a_count',b_follows_a_count
#        print 'a_usage_count',a_usage_count 
#        print  'b_usage_count',b_usage_count
#        print 'b_utterance_count',b_utterance_count 
#        print '----------------------'
        if (a_usage_count == 0 or b_utterance_count == 0): 
            coordination = -10 # invalid
        else:
            coordination = 1.0*b_follows_a_count/a_usage_count - 1.0*b_usage_count/b_utterance_count
        
        coordination_list[marker] = coordination
                # if ()
    
    return coordination_list
   
 
# test code : -10 indicates cannot be computed 
i = 0
for key in final_conversations.keys():    
    i +=1
    if (i > 40): break
        
    print compute_coordination(final_conversations[key])
    


{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': -10, 'personal_pronouns': -10, 'conjunctions': -10}
{'articles': -10, 'personal_pronouns': -10, 'conjunctions': -10}
{'articles': 0.5, 'personal_pronouns': 0.5, 'conjunctions': 0.0}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': -0.5, 'personal_pronouns': -0.5, 'conjunctions': -0.5}
{'articles': -10, 'personal_pronouns': -10, 'conjunctions': -10}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': -1.0, 'personal_pronouns': 0.0, 'conjunctions': -0.5}
{'articles': 0.0, 'personal_pronouns': 0.0, 'conjunctions': 0.0}
{'articles': -0.3333